In [1]:
# import requests
# from bs4 import BeautifulSoup
# import re
# import pandas as pd

# headers = {
#     # 'Accept': '',
#     # 'Accept-Encoding': '',
#     # 'Accept-Language': '',
#     # 'Cache-Control': '',
#     # 'Connection': '',
#     # 'Cookie':'',
#     # 'Host': '',
#     # 'Sec-Fetch-Mode': '',
#     # 'Sec-Fetch-Site': '',
#     # 'Sec-Fetch-User': '',
#     # 'Upgrade-Insecure-Requests': '',
#     # 'User-Agent': ''
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Host': 'store.steampowered.com',
#     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15',
#     'Accept-Language': 'en-US,en;q=0.9',
#     'Referer': 'https://store.steampowered.com/search/?term=of',
#     'Connection': 'keep-alive'
# }
# #替换你自己的headers
# n = 5
# #n代表爬取到多少页
# path = '1.xlsx'
# #修改你的保存位置

# def getgamelist(n):
#     linklist=[]
#     IDlist = []
#     for pagenum in range(n):
#         r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
#         soup = BeautifulSoup(r.text)
#         soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
#         for i in soups:
#             i = i.attrs
#             i = i['href']
#             link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
#             ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
#             linklist.append(link)
#             IDlist.append(ID)
#         print('已完成'+str(pagenum+1)+'页,目前共'+str(len(linklist)))
#     return linklist,IDlist

# def getdf(n):#转df
#     linklist,IDlist = getgamelist(n)
#     df = pd.DataFrame(list(zip(linklist,IDlist)),
#                columns =['Link', 'ID'])
#     return df
# if __name__ == "__main__":
#     df = getdf(n)#n代表爬取到多少页
#     df.to_csv(path)#储存

# Get game list

In [22]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os.path
import json
import datetime
n=404
def getList(n):
    linklist=[]
    IDlist=[]
    for page in range(n):
        soup = BeautifulSoup(requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%page).text)
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"))
        for i in soups:
            i = i.attrs['href']
            res=re.search('https://store.steampowered.com/app/(\d+)/',i)
            linklist.append(res.group(0))
            IDlist.append(res.group(1))
    return linklist,IDlist
if os.path.isfile('gamelist.csv'): #cache gamelist
    df = pd.read_csv('gamelist.csv')
else:
    linklist,IDlist=getList(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)), columns =['Link', 'ID'])
    df.to_csv('gamelist.csv')

# Get game data

In [27]:
def getPrice(soup):
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .game_purchase_price")
    if len(originalPrices)>0:
        for i in range(len(originalPrices)):
            if re.search('\$',originalPrices[i].string):
                originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
                return originalPrice, originalPrice
            elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
                return 0.,0.
    originalPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_original_price")
    discountPrices = soup.select(".game_area_purchase_game .game_purchase_action .discount_final_price")
    for i in range(len(originalPrices)):
        if re.search('\$',originalPrices[i].string):
            originalPrice = float(re.search('\$([\d.]+)',originalPrices[i].string).group(1))
            discountPrice = float(re.search('\$([\d.]+)',discountPrices[i].string).group(1))
            return originalPrice, discountPrice
        elif re.search('Free',originalPrices[i].string, re.IGNORECASE):
            return 0.,0.

def getReview(soup):
    try:
        review = str.strip(str(soup.find_all(class_="nonresponsive_hidden responsive_reviewdesc")[-1].contents[0]))
        rating = re.search('(\d+)\% of the ([\d,]+) user', review).group(1)
        rating = int(rating)
        reviewNum = re.search('(\d+)\% of the ([\d,]+) user', review).group(2)
        reviewNum = int(reviewNum.replace(',',''))
        return rating, reviewNum
    except:
        return None, 0

if not os.path.isfile('gameData.json'):
    gameFile = open('gameData.json','w')
    gameData = {}
    gameFile.close()
else:
    gameFile = open('gameData.json','r')
    try:
        gameData = json.load(gameFile)
    except:
        print('empty file')
        gameData = {}
    gameFile.close()
for p in range(1200):
    if p%20==0:
        print(p)
    if str(df['ID'][p]) not in gameData.keys():
        soup = BeautifulSoup(requests.get(df['Link'][p]).text)
        name = str(soup.find(class_="apphub_AppName").string)
        try:
            date = str(soup.find(class_="date").string)
        except:
            date = None
        try:
            originalPrice, discountPrice = getPrice(soup)
        except:
            originalPrice, discountPrice = 0.,0.
        rating, reviewNum = getReview(soup)
        description = str.strip(str(soup.find(class_="game_description_snippet").string))
        gameData[str(df['ID'][p])] = [name,date,originalPrice,discountPrice,rating,reviewNum,description]
gameFile = open('gameData.json','w')
gameFile.write(json.dumps(gameData))
gameFile.close()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180


In [19]:
print(gameData)

{'1938090': ['Call of Duty®: Modern Warfare® II', 'Oct 27, 2022', 69.99, 69.99, 62, 119361, 'Call of Duty®: Modern Warfare® II drops players into an unprecedented global conflict that features the return of the iconic Operators of Task Force 141.'], '1811260': ['EA SPORTS™ FIFA 23', 'Sep 29, 2022', 69.99, 41.99, 47, 25644, 'Experience the excitement of the biggest tournament in football with EA SPORTS™ FIFA 23 and the men’s FIFA World Cup™ update, available on November 9 at no additional cost!'], '570': ['Dota 2', 'Jul 9, 2013', 0.0, 0.0, 82, 1878389, "Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has taken on a life of its own."], '1174180': ['Red Dead Redemption 2', 'Dec 5, 2019', 59.99, 19.79, 89, 286676, 'Winner of over 175 Game of the Year Awa

In [5]:
x = pd.to_datetime('Oct 27, 2022')
x

Timestamp('2022-10-27 00:00:00')